In [1]:
from dash import html, dcc, Dash
from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd

THRESHOLD = 0.65

new_trialers = pd.read_csv(
        "../../data/predictions/08-14-22_1640_predictions.csv")
new_trialers["color"] = new_trialers.ChurnProb.apply(
    lambda x: "green" if x < THRESHOLD else "red")

fig = px.strip(
    new_trialers, 
    x='ChurnProb', 
    facet_row='trial_length_days',
    labels={"trial_length_days": "Length of Trial",
            "ChurnProb": "Probability of Churning"},
    color='color')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
server = app.server

In [2]:
app.layout = html.Div(
    [
        html.H1(
            'Skillshare Churn Predictions',
            style={
                'textAlign': 'center',
                'color': 'white',
                "background": "navy"}),
        dcc.Graph(
            id='graph-1',
            figure=fig)
    ], 
    style={
        "background": "#000080"})

In [3]:
app.run_server(mode='inline', debug=True, use_reloader=False)